# Sử dụng index với Dot product metric

In [1]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
import os
from dotenv import load_dotenv
load_dotenv('.env')
from transformers import BertTokenizerFast
import pinecone_text
from sentence_transformers import SentenceTransformer
import torch
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_API_ENV = os.environ['PINECONE_API_ENV']

d:\Work-Programe\Python\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Read data

In [2]:
def data_loader(file_sets, ):
    # Đọc dữ liệu từ các file .csv và trả lại toàn bộ dữ liệu
    sum_data = []
    for file_path in file_sets:
        loader = CSVLoader(file_path=f'./{file_path}',
                           
                            encoding='UTF-8')
        data = loader.load()
        sum_data+=data
    return sum_data
file_sets = ['Question_Sheet1.csv']
data = data_loader(file_sets)
for item in data[:5]:
    print(item.page_content)
# data

CÂU HỎI GỐC: Danh mục sản phẩm an toàn thông tin được nhập khẩu theo giấy phép?
CÂU HỎI GỐC: Nguyên tắc quản lý nhập khẩu sản phẩm an toàn thông tin mạng?
CÂU HỎI GỐC: Hệ thống thông tin quan trọng quốc gia trước khi đưa vào vận hành, khai thác phải đáp ứng yêu cầu gì về an toàn thông tin?
CÂU HỎI GỐC: Chủ quản hệ thống thông tin sử dụng ngân sách nhà nước có trách nhiệm gì?
CÂU HỎI GỐC: Các thiết bị đầu cuối có phải đối tượng giám sát an toàn thông tin không?


In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 150, chunk_overlap = 0)
texts = text_splitter.split_documents(data)
for item in texts[:2]:
    print(item.page_content)

CÂU HỎI GỐC: Danh mục sản phẩm an toàn thông tin được nhập khẩu theo giấy phép?
CÂU HỎI GỐC: Nguyên tắc quản lý nhập khẩu sản phẩm an toàn thông tin mạng?


In [13]:
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "index1" # put in the name of your pinecone index here

In [15]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [16]:
query = "định nghĩa về hệ thống thông tin"
docs = docsearch.similarity_search(query, include_metadata=True)
docs

[Document(page_content='CÂU HỎI GỐC: Định nghĩa phân loại cấp độ an toàn hệ thống thông tin', metadata={}),
 Document(page_content='CÂU HỎI GỐC: Hệ thống thông tin là gì?', metadata={}),
 Document(page_content='CÂU HỎI GỐC: Quy định về cấp độ hệ thống thông tin?', metadata={}),
 Document(page_content='CÂU HỎI GỐC: Hệ thống cơ sở hạ tầng thông tin?', metadata={})]